In [1]:
# Import important packages
import tensorflow as tf

### TestCase: Check for the parameter numbers for each 1D-CNN blocks and proceed with deciding their total number

In [2]:
# simplifying importing layers
from keras import layers

In [3]:
InputLayer = layers.Input(shape=(1000,12))

conv1d = layers.Conv1D(filters = 12, 
                                kernel_size = 5, # Usual default
                                strides = 1, # Usual default
                                activation = 'relu')(InputLayer)
MP1d = layers.MaxPool1D(pool_size = 2)(conv1d) # referenced usual default for ptbxl
DropOut = layers.Dropout(0.5)(MP1d) # referenced usual default for ptbxl
OutputLayer = layers.Dense(units = 6,# referenced usual default for ptbxl
                              activation = "softmax",# referenced usual default for ptbxl
                              )(DropOut)

FinalModel = tf.keras.Model(inputs = InputLayer,
                            outputs = OutputLayer,
                            name = "TestCaseModel_Conv_Block")

- The block above contains three hyperparameter tuning layers for more control over future instances.
- Apart from that, numbers of blocks needed to be implemented is still a long shot.

In [4]:
FinalModel.summary()

Model: "TestCaseModel_Conv_Block"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1000, 12)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 996, 12)        │           732 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 498, 12)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 498, 12)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 498, 6)         │            78 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 810 (3.16 KB)

 Trainable params: 810 (3.16 KB)

 Non-trainable params: 0 (0.00 B)

- Considering the total number of trainable parameters, we need to do some modifications and research to find if the number makes sense.
-  Designing Bi-LSTM layer is not much of a concern since most of the idea of their efficiency is negligible and its worth relying on the trail and error ideology to test which architecture is actually reliable.
- Consider using dropouts:
    - before output fully connected layer.
    - after 1D CNN layer and before the flatten layer in 1D CNN block (if implemented) 
    - after flatten layer in the classification overhead
- Consider using Batch Normalisation layers:
    - After the 1D convolution layer.
- Note: **Avoid using batch normalisation in RNN blocks!**

In [25]:
# create 2 1D CNN blocks -> see learnable parameters

# Input layers
InputLayer = layers.Input(shape=(1000,12))

# 1D-CNN Block 1
conv1d_1 = layers.Conv1D(filters = 12, 
                                kernel_size = 5, # Usual default
                                strides = 1, # Usual default
                                activation = "relu")(InputLayer)
BN_1 = layers.BatchNormalization()(conv1d_1)
MP1d_1 = layers.MaxPool1D(pool_size = 2)(BN_1) # referenced usual default for ptbxl
DropOut_1 = layers.Dropout(0.5)(MP1d_1) # referenced usual default for ptbxl

# 1D-CNN Block 2
conv1d_2 = layers.Conv1D(filters = 12, 
                                kernel_size = 5, # Usual default
                                strides = 1, # Usual default
                                activation = "relu")(DropOut_1)
BN_2 = layers.BatchNormalization()(conv1d_2)
MP1d_2 = layers.MaxPool1D(pool_size = 2)(BN_2) # referenced usual default for ptbxl
DropOut_2 = layers.Dropout(0.5)(MP1d_2)

# Bidirectional LSTM Block 1
Bidirectional_1 = layers.Bidirectional(
    layers.LSTM(64, return_sequences = True))(DropOut_2)

# Bidirectional LSTM Block 2
Bidirectional_2 = layers.Bidirectional(
    layers.LSTM(32))(Bidirectional_1)

# Classification Overhead
Flatten_2 = layers.Flatten()(Bidirectional_2) # referenced usual default for ptbxl
Dense_2 = layers.Dense(units = 64,
                      activation = "relu")(Flatten_2)
DropOut_Classification = layers.Dropout(0.5)(Dense_2)
OutputLayer = layers.Dense(units = 6,# referenced usual default for ptbxl
                              activation = "softmax",# referenced usual default for ptbxl
                              )(DropOut_Classification)

# Model Creation
FinalModel_two_blocks = tf.keras.Model(inputs = InputLayer,
                            outputs = OutputLayer,
                            name = "TestCaseModel_Conv_2_Blocks")

# Model Compilation (loss function and optimiser) 
FinalModel_two_blocks.compile(optimizer="adam",
                              loss = "CategoricalCrossentropy",
                              metrics = ["accuracy"])

In [26]:
FinalModel_two_blocks.summary()

Model: "TestCaseModel_Conv_2_Blocks"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 1000, 12)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_21 (Conv1D)              │ (None, 996, 12)        │           732 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 996, 12)        │            48 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_21 (MaxPooling1D) │ (None, 498, 12)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 498, 12)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_22 (Conv1D)              │ (None, 494, 12)        │           732 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 494, 12)        │            48 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_22 (MaxPooling1D) │ (None, 247, 12)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 247, 12)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_18                │ (None, 247, 128)       │        39,424 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_19                │ (None, 64)             │        41,216 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 86,750 (338.87 KB)

 Trainable params: 86,702 (338.68 KB)

 Non-trainable params: 48 (192.00 B)